In [1]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 27.1 MB/s eta 0:00:00


In [2]:
!pip show datasets

Name: datasets
Version: 2.13.1
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dill, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: evaluate


In [4]:
from google.colab import drive
import os
import numpy as np

# NOTE: this part is not needed if not running on collab
drive.mount('/content/drive')

# NOTE: navigate to the folder with dataset
folder = '/content/drive/My Drive/CS4NLP'
os.chdir(folder)

Mounted at /content/drive


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("CUDA is available")
else:
    device = torch.device("cpu")
    print("CUDA is not available, using CPU")

CUDA is available


In [ ]:
# install sentence transformers and get one model
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

#all-MiniLM-L6-v2 had 52, 47 accuracy
#multi-qa-mpnet-base-dot-v1 finetuned model on semantic search
sentembb_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1',  device='cuda') # sentence embedding model https://www.sbert.net/docs/usage/semantic_textual_similarity.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=e3fce1f3e7dc71e68769a2e9574352fd23a48aa0c9c4dd12cc47d749d95785b7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip show

In [5]:
from datasets import load_dataset, Dataset, load_from_disk
ds_dev = load_from_disk('datasets/quality/dev')
print(f"size of ds_dev:{len(ds_dev)}")

size of ds_dev:2086


In [ ]:
def split_text_into_chunks(text, max_token_size):
    text = text.replace("\n", "") # replace linebreaks
    sentences = text.split(". ") # split sentences
    sentences = [string for string in sentences if string] # remove empty strings ""
    sentences = [sentence if sentence.endswith(".") else sentence + ". " for sentence in sentences] # last sentence usually just ends with "." instead of ". ", do not add delimiter for them

    chunks = []
    current_chunk = ""
    current_chunk_size = 0

    for sentence in sentences:
        sentence_size =  len(sentence.split())
        if current_chunk_size +sentence_size < max_token_size:
            current_chunk += sentence
            current_chunk_size += sentence_size
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
            current_chunk_size = sentence_size

    if current_chunk: # add last element
        chunks.append(current_chunk)

    return chunks

In [ ]:
# generate embeddings
import pandas as pd

max_token_size = 32 # TODO: change to what you want

ds_sentembb_dict = {'article_id': [], 'chunk_id': [], 'chunk_text': [], 'sim_score': [], 'embedding':[]}

for question_id in range(0, len(ds_dev)):
  row = ds_dev[question_id]
  article = row['article']
  article = article.replace("\n", "") # replace linebreaks
  chunks = split_text_into_chunks(article, max_token_size)

  corpus_embeddings = sentembb_model.encode(chunks, convert_to_tensor=True)

  question = row['question']
  query_embedding = sentembb_model.encode(question, convert_to_tensor=True)

  cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0].cpu().numpy()

  if question_id % 100 == 0: # output
    print(f'{question_id}/{len(ds_dev)}')

  for chunk_id in range(0, len(chunks)):
    #new_row = {'article_id': 0, 'chunk_id': chunk_id, 'chunk_text': chunks[chunk_id], 'embedding': corpus_embeddings[chunk_id].cpu().numpy()}
    ds_sentembb_dict['article_id'].append(question_id)
    ds_sentembb_dict['chunk_id'].append(chunk_id)
    ds_sentembb_dict['chunk_text'].append(chunks[chunk_id])
    ds_sentembb_dict['sim_score'].append(cos_scores[chunk_id])
    ds_sentembb_dict['embedding'].append(corpus_embeddings[chunk_id].cpu().numpy())


sent_emb_df = pd.DataFrame.from_dict(ds_sentembb_dict)
print(sent_emb_df)

#sent_emb_df.to_csv(f'Chunkscores_tokensize_{str(max_token_size)}.csv')
sent_emb_df.to_pickle(f'Chunkscores_tokensize_{str(max_token_size)}.pickle')

In [ ]:
df = pd.read_pickle('Chunkscores_tokensize_256.pickle')
print(df[df['article_id']==0])